In [150]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'
SEED = 9496461801973866405 # this works
# SEED = 3158761121381184149

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]


if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)

tm = TsetlinMachine(train_x.shape[1], 5)
out_1 = tm.forward(train_x)
tm.l1.out

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


tensor([[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1]])

In [151]:
from tabulate import tabulate

zero_Y_row_idxs_per_W_row = []
for i in range(tm.l1.W.shape[0]):
    row_Y = tm.l1.out[:, i]
    zero_Y_idxs = torch.nonzero(row_Y == 0).squeeze(1).tolist()
    zero_Y_row_idxs_per_W_row.append(zero_Y_idxs)
zero_Y_row_idxs_per_W_row

[[0, 1, 2, 4, 5, 6, 7],
 [0, 2, 3, 4, 5, 6, 7],
 [0, 1, 3, 4, 6, 7],
 [0, 1, 2, 4, 5, 6],
 [0, 1, 2, 3, 4, 5, 6]]

In [152]:
from tabulate import tabulate

one_Y_row_idxs_per_W_row = []
for i in range(tm.l1.W.shape[0]):
    row_Y = tm.l1.out[:, i]
    zero_Y_idxs = torch.nonzero(row_Y == 1).squeeze(1).tolist()
    one_Y_row_idxs_per_W_row.append(zero_Y_idxs)
one_Y_row_idxs_per_W_row

[[3], [1], [2, 5], [3, 7], [7]]

In [153]:
tm.l1.W, tm.l1.full_X

(tensor([[1, 1, 1, 0, 0, 0],
         [1, 0, 1, 0, 1, 0],
         [0, 1, 0, 0, 0, 1],
         [0, 1, 1, 0, 0, 0],
         [0, 1, 1, 1, 0, 0]]),
 tensor([[1, 0, 0, 0, 1, 1],
         [1, 0, 1, 0, 1, 0],
         [1, 1, 0, 0, 0, 1],
         [1, 1, 1, 0, 0, 0],
         [0, 0, 0, 1, 1, 1],
         [0, 1, 0, 1, 0, 1],
         [0, 0, 1, 1, 1, 0],
         [0, 1, 1, 1, 0, 0]]))

In [154]:
tracking = {i:set(x) for i, x in enumerate(zero_Y_row_idxs_per_W_row)}
solution = {i:[0,0,0] for i in range(len(zero_Y_row_idxs_per_W_row))}

def funca():
    cols = []
    for i in range(tm.l1.in_dim):
        one_sets_occurrence_in_zeros = {x:[] for x in tracking.keys()}

        for k_1 in tracking.keys():
            for k_2 in tracking.keys():
                if k_1 != k_2:
                    if set(one_Y_row_idxs_per_W_row[k_1]).issubset(tracking[k_2]):
                        one_sets_occurrence_in_zeros[k_1].append(k_2)

        max_occurrence_row_idx = None
        max_occurrence_count = None
        count_dict = {k: len(v) for k,v in one_sets_occurrence_in_zeros.items()}
        for k,v in count_dict.items():
            if max_occurrence_row_idx is None or v > max_occurrence_count or (v == max_occurrence_count and len(one_Y_row_idxs_per_W_row[k]) > len(one_Y_row_idxs_per_W_row[max_occurrence_row_idx])):
                max_occurrence_count = v
                max_occurrence_row_idx = k

        one_rows = set(one_Y_row_idxs_per_W_row[max_occurrence_row_idx])
        corresponding_zeroes = tracking[max_occurrence_row_idx]
        zero_rows = set().union(*[one_Y_row_idxs_per_W_row[x] for x in one_sets_occurrence_in_zeros[max_occurrence_row_idx]])
        
        match_col_idx = -1
        match_col_side = None
        max_zero_rows_intersection = -1
        for i, col in enumerate(cols):
            # TODO: it doesnt have to be an exact subset, but something that could be made into a subset
            if one_rows.issubset(col[0]) and len(corresponding_zeroes & col[0]) == 0:
                if match_col_idx == -1 or len(zero_rows & col[1]) > max_zero_rows_intersection:
                    match_col_idx = i
                    max_zero_rows_intersection = len(zero_rows & col[1])
                    match_col_side = 0
            # elif one_rows.issubset(col[1]) and len(corresponding_zeroes & col[1]) == 0:
            #     if match_col == -1 or len(zero_rows & col[0]) > max_zero_rows_intersection:
            #         match_col = i
            #         max_zero_rows_intersection = len(zero_rows & col[0])
            #         match_one_rows_side = 1

        if match_col_idx == -1:
            left_side = one_rows
            right_side = zero_rows
            cols.append([one_rows, zero_rows])
            match_col_idx = len(cols) - 1
            match_col_side = 0
        else:
            match_col = cols[match_col_idx]
            allowed_zeroes = zero_rows - match_col[match_col_side]
            new_zero_rows = match_col[1- match_col_side] | allowed_zeroes
            match_col[1- match_col_side] = new_zero_rows

            left_side = match_col[0]
            right_side = match_col[1]
            
        remove_ks = []
        for k,v in tracking.items():
            one_value = set(one_Y_row_idxs_per_W_row[k])
            if one_value.issubset(left_side):
                sub = v - right_side
                if len(sub) > 0:
                    tracking[k] = sub
                    solution[k][match_col_idx] += 1
                else:
                    remove_ks.append(k)
                    solution[k][match_col_idx] += 1
            elif one_value.issubset(right_side):
                sub = v - left_side
                if len(sub) > 0:
                    solution[k][match_col_idx] -= 1
                    tracking[k] = sub
                else:
                    remove_ks.append(k)
                    solution[k][match_col_idx] -= 1
        for k in remove_ks:
            tracking.pop(k)
    return cols

cols = funca()
cols, solution, tracking

([[{2, 5}, {1, 3, 7}], [{1}, {3, 7}], [{3}, {7}]],
 {0: [-1, -1, 1],
  1: [-1, 1, 0],
  2: [1, 0, 0],
  3: [-1, -1, 0],
  4: [-1, -1, -1]},
 {0: {0, 4, 6}, 1: {0, 4, 6}, 2: {0, 4, 6}, 3: {0, 4, 6}, 4: {0, 4, 6}})

In [155]:
one_row_to_col_idxs = {}
available_rows_per_col = []
for col in cols:
    all_rows = col[0] | col[1]
    remaining_rows = set(range(tm.l1.full_X.shape[0])) - all_rows
    available_rows_per_col.append(remaining_rows)


for j, x in enumerate(one_Y_row_idxs_per_W_row):
    for i, col in enumerate(cols):
        if set(x).issubset(col[0]):
            if j not in one_row_to_col_idxs:
                one_row_to_col_idxs[j] = []
            one_row_to_col_idxs[j].append((i,0))

        elif set(x).issubset(col[1]):
            if j not in one_row_to_col_idxs:
                one_row_to_col_idxs[j] = []
            one_row_to_col_idxs[j].append((i,1))

col_idxs_to_one_rows = {}
for k,v in one_row_to_col_idxs.items():
    for col_idx in v:
        if col_idx not in col_idxs_to_one_rows:
            col_idxs_to_one_rows[col_idx] = []
        col_idxs_to_one_rows[col_idx].append(k)


sorted_keys = sorted(col_idxs_to_one_rows, key=lambda x: len(col_idxs_to_one_rows[x]), reverse=True)
for key in sorted_keys:
    one_rows = [x for x in col_idxs_to_one_rows[key] if x in tracking]
    if one_rows:

        intersect = set(range(tm.l1.full_X.shape[0])).intersection(*[tracking[x] for x in one_rows])
        intersect_with_available = intersect & available_rows_per_col[key[0]]

        available_rows_per_col[key[0]] -= intersect_with_available
        col_part = cols[key[0]][1-key[1]]
        col_part |= intersect_with_available
        remove_ks = []
        for k,v in tracking.items():
            if k in one_rows:
                sub = v - intersect_with_available
                if len(sub) > 0:
                    tracking[k] = sub
                else:
                    remove_ks.append(k)

        for k in remove_ks:
            tracking.pop(k)

remove_ks = []
for k,v in tracking.items():
    for i, col in enumerate(cols):
        col_remaining_size = tm.l1.full_X.shape[0]-(len(col[0]) + len(col[1]))
        one_clause = set(one_Y_row_idxs_per_W_row[k])
        if len(one_clause) <= col_remaining_size:
            if not one_clause.issubset(col[0]) and len(v & col[0]) == 0:
                col[0] |= one_clause
                col_remaining_size -= len(one_clause)
                remaining_v = list(v - col[1])
                remaining_to_add = remaining_v[:min(col_remaining_size, len(remaining_v))]
                remaining_v_size = len(remaining_v) - col_remaining_size
                col[1] |= set(remaining_to_add)
                if remaining_v_size > 0:
                    tracking[k] = sub
                    solution[k][i] += 1
                else:
                    remove_ks.append(k)
                    solution[k][i] += 1
                break
            elif not one_clause.issubset(col[1]) and len(v & col[1]) == 0:
                col[1] |= one_clause
                col_remaining_size -= len(one_clause)
                remaining_v = list(v - col[0])
                remaining_to_add = remaining_v[:min(col_remaining_size, len(remaining_v))]
                remaining_v_size = len(remaining_v) - col_remaining_size
                col[0] |= set(remaining_to_add)
                if remaining_v_size > 0:
                    tracking[k] = sub
                    solution[k][i] -= 1
                else:
                    remove_ks.append(k)
                    solution[k][i] -= 1
                break
for k in remove_ks:
    tracking.pop(k)
cols, tracking, solution

([[{0, 2, 4, 5, 6}, {1, 3, 7}], [{1, 2, 5}, {0, 3, 4, 6, 7}], [{3}, {7}]],
 {},
 {0: [-1, -1, 1],
  1: [-1, 1, 0],
  2: [1, 1, 0],
  3: [-1, -1, 0],
  4: [-1, -1, -1]})

In [156]:
for col in cols:
    remaining_rows = set(range(tm.l1.full_X.shape[0])) - (col[0] | col[1])
    col[1] |= remaining_rows
cols

[[{0, 2, 4, 5, 6}, {1, 3, 7}],
 [{1, 2, 5}, {0, 3, 4, 6, 7}],
 [{3}, {0, 1, 2, 4, 5, 6, 7}]]

In [157]:
new_full_X = torch.zeros_like(tm.l1.full_X)
for i, col in enumerate(cols):
    new_full_X[list(col[0]), i] = 1
    new_full_X[list(col[1]), i + tm.l1.in_dim] = 1
new_full_X


tensor([[1, 0, 0, 0, 1, 1],
        [0, 1, 0, 1, 0, 1],
        [1, 1, 0, 0, 0, 1],
        [0, 0, 1, 1, 1, 0],
        [1, 0, 0, 0, 1, 1],
        [1, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1, 1]])

In [158]:
new_W = torch.zeros_like(tm.l1.W)
for i, x in enumerate(one_Y_row_idxs_per_W_row):
    W_row_values = solution[i]
    for j, val in enumerate(W_row_values):
        if val > 0:
            new_W[i, j] = 1
        elif val < 0:
            new_W[i, j + tm.l1.in_dim] = 1
new_W

tensor([[0, 0, 1, 1, 1, 0],
        [0, 1, 0, 1, 0, 0],
        [1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1]])

In [159]:
from tsetlin import TsetlinBase

tb = TsetlinBase()
y2 = tb.conjunction_mul(new_full_X.unsqueeze(1), new_W)
y2

tensor([[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1]])